<h1 id="Contents">Contents<a href="#Contents"></a></h1>
        <ol>
        <li><a class="" href="#Imports">Imports</a></li>
<li><a class="" href="#Animation">Animation</a></li>
<ol><li><a class="" href="#Simple-Replacements">Simple Replacements</a></li>
<li><a class="" href="#Transformations">Transformations</a></li>
<li><a class="" href="#Animation-Syntax">Animation Syntax</a></li>
<ol><li><a class="" href="#Animation-Running-Time">Animation Running Time</a></li>
<li><a class="" href="#Creating-a-custom-animation">Creating a custom animation</a></li>
</ol><li><a class="" href="#Updaters">Updaters</a></li>
<li><a class="" href="#ValueTracker">ValueTracker</a></li>
</ol><li><a class="" href="#Some-More-Considerations">Some More Considerations</a></li>
<ol><li><a class="" href="#Frame-Rate">Frame Rate</a></li>
<li><a class="" href="#Rotation-Animation">Rotation Animation</a></li>
</ol>

# Imports

In [38]:
from manim import *
config.media_embed = True

In [39]:
#ignore this cell, only for setup
param= "-v WARNING  --progress_bar None  -r  500,300  --disable_caching Example"

param_progress = "-v WARNING -r  500,300  --disable_caching Example"

NO  = Cross(Square(), stroke_color = RED_D, stroke_width = 38).scale(0.9).to_edge(LEFT, buff=1)
YES = SVGMobject("tick.svg").to_edge(LEFT, buff=1)
BEST = YES.copy()
BEST.add(Star(color= YELLOW, fill_opacity=1).scale(0.5).move_to(BEST).shift(0.5*DOWN+0.5*RIGHT));

[Reference 1](https://flyingframes.readthedocs.io/en/latest/ch4.html)

[Reference 2](https://docs.manim.community/en/stable/tutorials/building_blocks.html)

[Reference 3](https://slama.dev/manim/groups-transformations-updaters/#updaters)

# Animation

Let's now discuss the very thing manim was made for, animations!

There are a wide range of possibilities to animate your mobjects that all work a bit differently. In this notebook, we'll discuss some of them.

## Simple Replacements

The simplest type of animation is to just use replacement, like this:

In [40]:
%%manim $param
class Example(Scene):
    def construct(self):
        dot= Dot(color= YELLOW, radius=0.5)
        self.add(dot)
        self.wait()
        dot.scale(2)
        self.wait()
        dot.scale(2)
        self.wait(2)

Or like this:

In [41]:
%%manim $param
class Example(Scene):
    def construct(self):
        dot= Dot(color= YELLOW, radius=0.5)
        square= Square(side_length=4,color= BLUE, fill_opacity=1)
        triangle= Triangle(radius=3,color= ORANGE, fill_opacity=1).shift(DOWN*0.5)
        self.add(dot)
        self.wait()
        dot.become(square)
        self.wait()
        dot.become(triangle)
        self.wait()

But it looks ugly, isn't it? Let's make it look better by using `animate` syntax.

## Transformations

This is another way to animate stuffs. The follwoing code shows an example:

In [42]:
%%manim $param
class Example(Scene):
    def construct(self):
        d= Dot(color= YELLOW, radius=0.5)
        d2= d.copy().shift(2*RIGHT)
        self.play(Transform(d, d2))

## Animation Syntax

You can use the `.animate` mathod of a mobject to animate it.

In [43]:
%%manim $param
class Example(Scene):
    def construct(self):
        dot= Dot(color= YELLOW, radius=0.5)
        self.play(dot.animate.scale(2))

You can also use the `self.play` method to animate a mobject.

In [44]:
%%manim $param

class Example(Scene):
    def construct(self):
        square = Square()

        # some animations display mobjects, ...
        self.play(FadeIn(square))

        # ... some move or rotate mobjects around...
        self.play(Rotate(square, PI/4))

        # some animations remove mobjects from the screen
        self.play(FadeOut(square))

        self.wait(1)

Put simply, animations are procedures that interpolate between two mobjects. For example, `FadeIn(square)` starts with a fully transparent version of square and ends with a fully opaque version, interpolating between them by gradually increasing the opacity. `FadeOut` works in the opposite way: it interpolates from fully opaque to fully transparent. As another example, `Rotate` starts with the mobject passed to it as argument, and ends with the same object but rotated by a certain amount, this time interpolating the mobject’s angle instead of its opacity.

Any property of a mobject that can be changed can be animated. In fact, any method that changes a mobject’s property can be used as an animation, through the use of `animate()`. This is where the true power of manim lies: you can animate any property of any mobject, and you can even create your own animations. More on this later.

In [45]:
%%manim $param

class Example(Scene):
    def construct(self):
        square = Square().set_fill(RED, opacity=1.0)
        self.add(square)

        # animate the change of color
        self.play(square.animate.set_fill(WHITE))
        self.wait(1)

        # animate the change of position and the rotation at the same time
        self.play(square.animate.shift(UP).rotate(PI / 3))
        self.wait(1)

`animate()` is a property of all mobjects that animates the methods that come afterward. For example, `square.set_fill(WHITE)` sets the fill color of the square, while `square.animate.set_fill(WHITE)` animates this action.

### Animation Running Time

By default, any animation passed to `play()` lasts for exactly one second. Use the `run_time` argument to control the duration.

In [46]:
%%manim $param

class Example(Scene):
    def construct(self):
        square = Square()
        self.add(square)
        self.play(square.animate.shift(UP), run_time=3)
        self.wait(1)

### Creating a custom animation

Even though Manim has many built-in animations, you will find times when you need to smoothly animate from one state of a `Mobject` to another. If you find yourself in that situation, then you can define your own custom animation. You start by extending the Animation class and overriding its `interpolate_mobject()`. The `interpolate_mobject()` method receives `alpha` as a parameter that starts at 0 and changes throughout the animation. So, you just have to manipulate `self.mobject` inside Animation according to the `alpha` value in its interpolate_mobject method. Then you get all the benefits of `Animation` such as playing it for different run times or using different rate functions.

Let’s say you start with a number and want to create a `Transform` animation that transforms it to a target number. You can do it using `FadeTransform`, which will fade out the starting number and fade in the target number. But when we think about transforming a number from one to another, an intuitive way of doing it is by incrementing or decrementing it smoothly. Manim has a feature that allows you to customize this behavior by defining your own custom animation.

The alpha parameter holds a value between 0 and 1 representing the step of the currently playing animation. For example, 0 means the beginning of the animation, 0.5 means halfway through the animation, and 1 means the end of the animation.

This means we need to figure out a way to determine the number to display at the given alpha value and then set that value in the `interpolate_mobject()` method of the Count animation. Suppose you are starting at 50 and incrementing until the `DecimalNumber` reaches 100 at the end of the animation.

- If alpha is 0, you want the value to be 50.

- If alpha is 0.5, you want the value to be 75.

- If alpha is 1, you want the value to be 100.

Let's write the code:

In [47]:
param

'-v WARNING  --progress_bar None  -r  500,300  --disable_caching Example'

In [48]:
%%manim -v WARNING  --progress_bar None  -r  500,300  --disable_caching CountingScene

class Count(Animation):
    def __init__(self, number: DecimalNumber, start: float, end: float, **kwargs) -> None:
        # Pass number as the mobject of the animation
        super().__init__(number,  **kwargs)
        # Set start and end
        self.start = start
        self.end = end

    def interpolate_mobject(self, alpha: float) -> None:
        # Set value of DecimalNumber according to alpha
        value = self.start + (alpha * (self.end - self.start))
        self.mobject.set_value(value)


class CountingScene(Scene):
    def construct(self):
        # Create Decimal Number and add it to scene
        number = DecimalNumber().set_color(WHITE).scale(5)
        # Add an updater to keep the DecimalNumber centered as its value changes
        number.add_updater(lambda number: number.move_to(ORIGIN))

        self.add(number)

        self.wait()

        # Play the Count Animation to count from 0 to 100 in 4 seconds
        self.play(Count(number, 0, 100), run_time=4, rate_func=linear)

        self.wait()

## Updaters

Imagine that we would like to continually update an object during an animation (its position, scale, etc.). This what Manim’s updaters are for – they are functions containing arbitrary code, connected to a certain object and evaluated each rendered frame.

Let's see a simple updater.

In [49]:
%%manim $param
class Example(Scene):
    def construct(self):
        dot = Dot(color= GREEN, radius=0.7)
        self.add(dot)
        def foo(mob,dt):
            mob.shift(1.4*RIGHT*dt)
        dot.add_updater(foo)
        self.wait(5)

For a more involved example, see this:

In [50]:
%%manim $param
class Example(Scene):
    def construct(self):
        square = Square()
        square_label = Tex("A neat square.").next_to(square, UP, buff=0.5)

        self.play(Write(square))
        self.play(FadeIn(square_label, shift=UP * 0.5))

        def label_updater(obj):
            """An updater which continually move an object above the square.

            The first parameter (obj) is always the object that is being updated."""
            obj.next_to(square, UP, buff=0.5)

        # we want the label to always reside above the square
        square_label.add_updater(label_updater)

        self.play(square.animate.shift(LEFT * 3))
        self.play(square.animate.scale(1 / 2))
        self.play(square.animate.rotate(PI / 2).shift(RIGHT * 3 + DOWN * 0.5).scale(3))

        # to pause updaters, we'll use suspend_updating()
        # square_label.suspend_updating()

        self.play(square.animate.scale(1 / 3))
        self.play(square.animate.rotate(PI / 2))

        # to resume,we'll use resume__updating()
        # square_label.resume_updating()

        self.play(square.animate.scale(3))
        self.play(square.animate.rotate(PI / 2))

In [51]:
%%manim $param
class Example(Scene):
    def construct(self):
        square = Square()
        square_label = Tex("A neat square.").next_to(square, UP, buff=0.5)

        self.play(Write(square))
        self.play(FadeIn(square_label, shift=UP * 0.5))

        def label_updater(obj):
            """An updater which continually move an object above the square.

            The first parameter (obj) is always the object that is being updated."""
            obj.next_to(square, UP, buff=0.5)

        # we want the label to always reside above the square
        square_label.add_updater(label_updater)

        self.play(square.animate.shift(LEFT * 3))
        self.play(square.animate.scale(1 / 2))
        self.play(square.animate.rotate(PI / 2).shift(RIGHT * 3 + DOWN * 0.5).scale(3))

        # to pause updaters, we'll use suspend_updating()
        square_label.suspend_updating()

        self.play(square.animate.scale(1 / 3))
        self.play(square.animate.rotate(PI / 2))

        # to resume,we'll use resume__updating()
        square_label.resume_updating()

        self.play(square.animate.scale(3))
        self.play(square.animate.rotate(PI / 2))

Besides changing certain attributes of an object, it might be useful to transform it to an entirely new one. We’ll use the `become` function, which transforms an object into another one immediately (not like `Transform`, which is an animation).

In [52]:
%%manim $param_progress

class Example(Scene):
    def format_point(self, point) -> str:
        """Format the given point to look presentable."""
        return f"[{point[0]:.2f}, {point[1]:.2f}]"

    def construct(self):
        circle = Circle(color=WHITE)

        def circle_label_updater(obj):
            """An updater that displays the circle's position above it."""
            obj.become(Tex(f"p = {self.format_point(circle.get_center())}"))
            obj.next_to(circle, UP, buff=0.35)

        self.play(Write(circle))

        circle_label = Tex()

        # a bit of a hack - we're calling the updater to create the initial label
        circle_label_updater(circle_label)

        self.play(FadeIn(circle_label, shift=UP * 0.3))

        # start updating the label
        circle_label.add_updater(circle_label_updater)

        self.play(circle.animate.shift(RIGHT))
        self.play(circle.animate.shift(LEFT * 3 + UP))
        self.play(circle.animate.shift(DOWN * 2 + RIGHT * 2))
        self.play(circle.animate.shift(UP))

## ValueTracker

A mobject that can be used for tracking (real-valued) parameters. Useful for animating parameter changes.

Not meant to be displayed. Instead the position encodes some number, often one which another animation or continual_animation uses for its update function, and by treating it as a mobject it can still be animated and manipulated just like anything else.

This value changes continuously when animated using the `animate` syntax. In short `ValueTracker` is a class let you get increasing/decreasing value while playing scene.

That is, let's suppose you want to animate number increasing from 0 to 1000 within 5 seconds animation, you can use ValueTracker as below:

In [53]:
%%manim $param
class Example(Scene):
    def construct(self):
        tracker = ValueTracker(0)
        number = DecimalNumber(0)
        number.scale(5)
        number.add_updater(lambda m:m.set_value(tracker.get_value()))

        self.add(number)
        self.play(tracker.animate.set_value(10), run_time = 5)
        self.wait()

We see that using a `ValueTracker`, we can create custom animation. See the section where we created the animation of number count by creating a custom animation.

Another plus point of using `ValueTracker` is that this way, the length of the animation becomes independent of the frame rate. Let's see some more examples:

In [54]:
%%manim $param
class Example(Scene):
    def construct(self):
        tracker= ValueTracker(0)
        dot = Dot(color= GREEN, radius=0.7)
        self.add(dot)
        def foo(mob):
            mob.move_to(RIGHT*tracker.get_value())
        dot.add_updater(foo)
        self.play(tracker.animate.set_value(2), rate_func= linear)

Note the use of `rate_func` parameter. We can try some more functions or we can even define custom ones.

In [55]:
%%manim $param
class Example(Scene):
    def construct(self):
        tracker= ValueTracker(0)
        dot = Dot(color= GREEN, radius=0.7)
        self.add(dot)
        def foo(mob):
            mob.move_to(RIGHT*tracker.get_value())
        dot.add_updater(foo)
        self.play(tracker.animate.set_value(2), rate_func= smooth)

In [56]:
%%manim $param

def low_frame_rate(t):
    return np.floor(t*10)/10

class Example(Scene):
    def construct(self):
        self.camera.background_color = "#ece6e2"
        banner = ManimBanner(dark_theme=False).scale(0.7)
        banner2 = banner.copy()
        Group(banner, banner2).arrange(DOWN)
        self.play(banner.expand(), rate_func = low_frame_rate)
        self.play(banner2.expand())
        self.wait()

Here, a custom rate function is defined which is for a low frame rate within a high frame rate video.

We can play with the `ValueTracker` to do more:

In [57]:
%%manim $param
class Example(Scene):
    def construct(self):
        tracker= ValueTracker(0.5)
        dot = Dot(color= GREEN, radius=0.7)
        self.add(dot)
        def foo(mob):
            mob.move_to(RIGHT*tracker.get_value())
        dot.add_updater(foo)
        self.play(tracker.animate.set_value(2.2), rate_func= smooth)
        self.play(tracker.animate.increment_value(1), rate_func= smooth)
        self.play(tracker.animate.increment_value(-1), rate_func= smooth)
        self.play(tracker.animate.set_value(0.5), rate_func= linear)

It’s also possible to use different rate functions simultaneously:

In [58]:
%%manim $param
class Example(Scene):
    def construct(self):
        line1 = Line(3*LEFT, 3*RIGHT, stroke_width=10).set_color(RED)
        line2 = Line(3*LEFT, 3*RIGHT, stroke_width=10).set_color(GREEN)
        Group(line1,line2).arrange(DOWN, buff=2)

        d1 = Dot(point = line1.get_left(), radius=0.5)
        d2 = Dot(point = line2.get_left(), radius=0.5)

        label1 = Tex("smooth").scale(2).next_to(line1, buff= 1)
        label2 = Tex("linear").scale(2).next_to(line2, buff= 1)

        tr1=ValueTracker(-3)
        tr2=ValueTracker(-3)

        d1.add_updater(lambda z: z.set_x(tr1.get_value()))
        d2.add_updater(lambda z: z.set_x(tr2.get_value()))

        self.add(line1,line2,d1,d2,label1,label2 )

        self.play(tr1.animate(rate_func=smooth).set_value(3), tr2.animate(rate_func=linear).set_value(3), run_time = 3)
        self.wait()

One can now also add additional properties to mobjects, in this case a counter:

In [59]:
%%manim $param
class Example(Scene):
    def construct(self):
        tracker= ValueTracker(0)
        dot = Dot(color= GREEN, radius=0.7)
        self.add(dot)
        dot.counter=0
        def foo(mob):
            mob.move_to(RIGHT*tracker.get_value())
            if mob.counter == 20:
                mob.set_color(random_bright_color())
                mob.counter = 0
            mob.counter += 1
        dot.add_updater(foo)
        self.play(tracker.animate.set_value(2), rate_func= linear, run_time=3)

# Some More Considerations

## Frame Rate

The frame rate of the video may cause problems sometimes. By default, frame rate is 60 fps, as can be seen by:

In [60]:
config.frame_rate

60.0

The problem can escpecially occur when using updaters. For example:

In [61]:
%%manim $param
class Example(Scene):
    def construct(self):
        dotred= Dot(color= RED, radius=0.5).shift(UP)
        dotgreen = Dot(color= GREEN, radius=0.5)
        dotgreen.next_to(dotred,DOWN)
        self.add(dotgreen,dotred)
        DIR= 2*RIGHT
        dotgreen.add_updater(lambda x,dt: x.shift(DIR*dt))
        dotred.add_updater(lambda x,dt: x.shift(DIR*1/60))
        self.wait(3)

Now try this:

In [62]:
param5fps = "-v WARNING  --progress_bar None  --frame_rate=5 -r  500,200  --disable_caching Example"

In [63]:
%%manim $param5fps
class Example(Scene):
    def construct(self):
        print(f"{config.frame_rate = }fps")
        dotred= Dot(color= RED, radius=0.5).shift(UP)
        dotgreen = Dot(color= GREEN, radius=0.5)
        dotgreen.next_to(dotred,DOWN)
        self.add(dotgreen,dotred)
        DIR= 2*RIGHT
        dotgreen.add_updater(lambda x,dt: x.shift(DIR*dt))
        dotred.add_updater(lambda x,dt: x.shift(DIR*1/60))
        self.wait(3)

config.frame_rate = 5.0fps


One way to avoid this is define you updaters appropriately. Another way is to use `ValueTrackers`.

In [64]:
%%manim $param5fps
class Example(Scene):
    def construct(self):
        print(f"{config.frame_rate = }fps")
        tracker = ValueTracker(0)
        dotred= Dot(color= RED, radius=0.5).shift(UP)
        dotgreen = Dot(color= GREEN, radius=0.5)
        dotgreen.next_to(dotred,DOWN)
        self.add(dotgreen,dotred)
        dotgreen.add_updater(lambda x: x.set_x(tracker.get_value()))
        dotred.add_updater(lambda x: x.set_x(tracker.get_value()))
        self.play(tracker.animate.set_value(3), rate = linear, run_time = 3)

config.frame_rate = 5.0fps


## Rotation Animation

There are multiple ways to rotate a square, but not all will result in that animation that you might have expected.



In [65]:
%%manim $param

class Example(Scene):
    def construct(self, **kwargs):
        s1= Square().set_color(YELLOW)
        self.add(s1, BEST)
        self.play(Rotate(s1, angle=PI/2))

In [66]:
%%manim $param
class Example(Scene):
    def construct(self, **kwargs):
        s2= Square().set_color(PURPLE)
        self.add(s2, NO)
        self.play(s2.animate.rotate(PI/2))
    

In [67]:
%%manim $param

class Example(Scene):
    def construct(self, **kwargs):
        theta_track= ValueTracker(0)
        s3= Square().set_color(ORANGE)
        self.add(s3, YES)
        s3.previous_angle=0
        def pref(x):
            x.previous_angle=theta_track.get_value()
        s3.add_updater(lambda x: x.rotate(theta_track.get_value()-s3.previous_angle))
        s3.add_updater(pref)
        self.play(theta_track.animate.increment_value(PI/2))

In [68]:
%%manim $param
class Example(Scene):
    def construct(self, **kwargs):
        s6= Square().set_color(PINK)
        self.add(s6, YES)
        s6.add_updater(lambda x, dt: x.rotate(dt*PI/2))
        self.wait(1)

And here is this "remarkable" rotation!

In [69]:
%%manim $param
# NOT WORKING!, BAD PRACTICE.
class Example(Scene):
    def construct(self, **kwargs):
        s4= Square().set_color(GREEN)
        self.add(s4, NO)
        theta_track= ValueTracker(0)
        s4.add_updater(lambda x: x.rotate(theta_track.get_value()))
        self.play(theta_track.animate.increment_value(PI/2))